# LangGraph and LangSmith - Agentic RAG Powered by LangChain

In the following notebook we'll complete the following tasks:

- 🤝 Breakout Room #1:
  1. Install required libraries
  2. Set Environment Variables
  3. Creating our Tool Belt
  4. Creating Our State
  5. Creating and Compiling A Graph!

- 🤝 Breakout Room #2:
  1. Evaluating the LangGraph Application with LangSmith
  2. Adding Helpfulness Check and "Loop" Limits
  3. LangGraph for the "Patterns" of GenAI

# 🤝 Breakout Room #1

## Part 1: LangGraph - Building Cyclic Applications with LangChain

LangGraph is a tool that leverages LangChain Expression Language to build coordinated multi-actor and stateful applications that includes cyclic behaviour.

### Why Cycles?

In essence, we can think of a cycle in our graph as a more robust and customizable loop. It allows us to keep our application agent-forward while still giving the powerful functionality of traditional loops.

Due to the inclusion of cycles over loops, we can also compose rather complex flows through our graph in a much more readable and natural fashion. Effectively allowing us to recreate application flowcharts in code in an almost 1-to-1 fashion.

### Why LangGraph?

Beyond the agent-forward approach - we can easily compose and combine traditional "DAG" (directed acyclic graph) chains with powerful cyclic behaviour due to the tight integration with LCEL. This means it's a natural extension to LangChain's core offerings!

## Task 1:  Dependencies


## Task 2: Environment Variables

We'll want to set both our OpenAI API key and our LangSmith environment variables.

In [1]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

In [2]:
os.environ["TAVILY_API_KEY"] = getpass.getpass("TAVILY_API_KEY")

In [3]:
from uuid import uuid4

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"AIE7 - LangGraph - {uuid4().hex[0:8]}"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangSmith API Key: ")

## Task 3: Creating our Tool Belt

As is usually the case, we'll want to equip our agent with a toolbelt to help answer questions and add external knowledge.

There's a tonne of tools in the [LangChain Community Repo](https://github.com/langchain-ai/langchain-community/tree/main/libs/community) but we'll stick to a couple just so we can observe the cyclic nature of LangGraph in action!

We'll leverage:

- [Tavily Search Results](https://github.com/langchain-ai/langchain-community/blob/main/libs/community/langchain_community/tools/tavily_search/tool.py)
- [Arxiv](https://github.com/langchain-ai/langchain-community/blob/main/libs/community/langchain_community/tools/arxiv/tool.py)

#### 🏗️ Activity #1:

Please add the tools to use into our toolbelt.

> NOTE: Each tool in our toolbelt should be a method.

In [4]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.tools.arxiv.tool import ArxivQueryRun

tavily_tool = TavilySearchResults(max_results=5)

tool_belt = [
    tavily_tool,
    ArxivQueryRun(),
]

/var/folders/kv/_x494dyj0z5gt4dzp94btrb80000gn/T/ipykernel_68905/1203815797.py:4: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tavily_tool = TavilySearchResults(max_results=5)


### Model

Now we can set-up our model! We'll leverage the familiar OpenAI model suite for this example - but it's not *necessary* to use with LangGraph. LangGraph supports all models - though you might not find success with smaller models - as such, they recommend you stick with:

- OpenAI's GPT-3.5 and GPT-4
- Anthropic's Claude
- Google's Gemini

> NOTE: Because we're leveraging the OpenAI function calling API - we'll need to use OpenAI *for this specific example* (or any other service that exposes an OpenAI-style function calling API.

In [6]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4.1-nano", temperature=0)

Now that we have our model set-up, let's "put on the tool belt", which is to say: We'll bind our LangChain formatted tools to the model in an OpenAI function calling format.

In [7]:
model = model.bind_tools(tool_belt)

#### ❓ Question #1:

How does the model determine which tool to use?

### Answer 
All tools inherit from the BaseTool class. Each tool definition must provide a description, as well as specify its input and output parameters. This metadata helps the model understand the purpose and usage of each tool, enabling it to select the most appropriate tool based on the user's query.

## Task 4: Putting the State in Stateful

Earlier we used this phrasing:

`coordinated multi-actor and stateful applications`

So what does that "stateful" mean?

To put it simply - we want to have some kind of object which we can pass around our application that holds information about what the current situation (state) is. Since our system will be constructed of many parts moving in a coordinated fashion - we want to be able to ensure we have some commonly understood idea of that state.

LangGraph leverages a `StatefulGraph` which uses an `AgentState` object to pass information between the various nodes of the graph.

There are more options than what we'll see below - but this `AgentState` object is one that is stored in a `TypedDict` with the key `messages` and the value is a `Sequence` of `BaseMessages` that will be appended to whenever the state changes.

Let's think about a simple example to help understand exactly what this means (we'll simplify a great deal to try and clearly communicate what state is doing):

1. We initialize our state object:
  - `{"messages" : []}`
2. Our user submits a query to our application.
  - New State: `HumanMessage(#1)`
  - `{"messages" : [HumanMessage(#1)}`
3. We pass our state object to an Agent node which is able to read the current state. It will use the last `HumanMessage` as input. It gets some kind of output which it will add to the state.
  - New State: `AgentMessage(#1, additional_kwargs {"function_call" : "WebSearchTool"})`
  - `{"messages" : [HumanMessage(#1), AgentMessage(#1, ...)]}`
4. We pass our state object to a "conditional node" (more on this later) which reads the last state to determine if we need to use a tool - which it can determine properly because of our provided object!

In [8]:
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages
import operator
from langchain_core.messages import BaseMessage

class AgentState(TypedDict):
  messages: Annotated[list, add_messages]

## Task 5: It's Graphing Time!

Now that we have state, and we have tools, and we have an LLM - we can finally start making our graph!

Let's take a second to refresh ourselves about what a graph is in this context.

Graphs, also called networks in some circles, are a collection of connected objects.

The objects in question are typically called nodes, or vertices, and the connections are called edges.

Let's look at a simple graph.

![image](https://i.imgur.com/2NFLnIc.png)

Here, we're using the coloured circles to represent the nodes and the yellow lines to represent the edges. In this case, we're looking at a fully connected graph - where each node is connected by an edge to each other node.

If we were to think about nodes in the context of LangGraph - we would think of a function, or an LCEL runnable.

If we were to think about edges in the context of LangGraph - we might think of them as "paths to take" or "where to pass our state object next".

Let's create some nodes and expand on our diagram.

> NOTE: Due to the tight integration with LCEL - we can comfortably create our nodes in an async fashion!

In [9]:
from langgraph.prebuilt import ToolNode

def call_model(state):
  messages = state["messages"]
  response = model.invoke(messages, config={"recursion_limit": 10})
  return {"messages" : [response]}

tool_node = ToolNode(tool_belt)

Now we have two total nodes. We have:

- `call_model` is a node that will...well...call the model
- `tool_node` is a node which can call a tool

Let's start adding nodes! We'll update our diagram along the way to keep track of what this looks like!


In [10]:
from langgraph.graph import StateGraph, END

uncompiled_graph = StateGraph(AgentState)

uncompiled_graph.add_node("agent", call_model)
uncompiled_graph.add_node("action", tool_node)

Let's look at what we have so far:

![image](https://i.imgur.com/md7inqG.png)

Next, we'll add our entrypoint. All our entrypoint does is indicate which node is called first.

In [11]:
uncompiled_graph.set_entry_point("agent")

![image](https://i.imgur.com/wNixpJe.png)

Now we want to build a "conditional edge" which will use the output state of a node to determine which path to follow.

We can help conceptualize this by thinking of our conditional edge as a conditional in a flowchart!

Notice how our function simply checks if there is a "function_call" kwarg present.

Then we create an edge where the origin node is our agent node and our destination node is *either* the action node or the END (finish the graph).

It's important to highlight that the dictionary passed in as the third parameter (the mapping) should be created with the possible outputs of our conditional function in mind. In this case `should_continue` outputs either `"end"` or `"continue"` which are subsequently mapped to the action node or the END node.

In [12]:
def should_continue(state):
  last_message = state["messages"][-1]

  if last_message.tool_calls:
    return "action"

  return END

uncompiled_graph.add_conditional_edges(
    "agent",
    should_continue
)

Let's visualize what this looks like.

![image](https://i.imgur.com/8ZNwKI5.png)

Finally, we can add our last edge which will connect our action node to our agent node. This is because we *always* want our action node (which is used to call our tools) to return its output to our agent!

In [13]:
uncompiled_graph.add_edge("action", "agent")

Let's look at the final visualization.

![image](https://i.imgur.com/NWO7usO.png)

All that's left to do now is to compile our workflow - and we're off!

In [14]:
simple_agent_graph = uncompiled_graph.compile()

#### ❓ Question #2:

Is there any specific limit to how many times we can cycle?

If not, how could we impose a limit to the number of cycles?

### Answer
1. Currently, there is no built-in limit to the number of cycles the agent can perform.
2. However, you can control this behavior by setting the `recursion_limit` parameter, which restricts the maximum number of cycles or iterations the agent is allowed to execute. https://langchain-ai.github.io/langgraph/concepts/low_level/#recursion-limit


## Using Our Graph

Now that we've created and compiled our graph - we can call it *just as we'd call any other* `Runnable`!

Let's try out a few examples to see how it fairs:

In [15]:
from langchain_core.messages import HumanMessage

inputs = {"messages" : [HumanMessage(content="Who is the current captain of the Winnipeg Jets?")]}

async for chunk in simple_agent_graph.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        print(values["messages"])
        print("\n\n")

Receiving update from node: 'agent'
[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_etP7wULe03MdPFEcOAu2sgnp', 'function': {'arguments': '{"query":"current captain of the Winnipeg Jets"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 162, 'total_tokens': 185, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': None, 'id': 'chatcmpl-BsVDsnFgflWUIFCPSdMiX0puwOybD', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--a32da42b-3b48-401f-89f2-36719beedcdb-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current captain of the Winnipeg Jets'}, 'id': 'call_etP7wULe03MdPFEcOAu2sgnp', 'type': 

Let's look at what happened:

1. Our state object was populated with our request
2. The state object was passed into our entry point (agent node) and the agent node added an `AIMessage` to the state object and passed it along the conditional edge
3. The conditional edge received the state object, found the "tool_calls" `additional_kwarg`, and sent the state object to the action node
4. The action node added the response from the OpenAI function calling endpoint to the state object and passed it along the edge to the agent node
5. The agent node added a response to the state object and passed it along the conditional edge
6. The conditional edge received the state object, could not find the "tool_calls" `additional_kwarg` and passed the state object to END where we see it output in the cell above!

Now let's look at an example that shows a multiple tool usage - all with the same flow!

In [17]:
inputs = {"messages" : [HumanMessage(content="Search Arxiv for the QLoRA paper, then search each of the authors to find out their latest Tweet using Tavily!")]}

async for chunk in simple_agent_graph.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        if node == "action":
          print(f"Tool Used: {values['messages'][0].name}")
        print(values["messages"])

        print("\n\n")

Receiving update from node: 'agent'
[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Wo1zscbYGJ1csSTZYwux7SS6', 'function': {'arguments': '{"query": "QLoRA"}', 'name': 'arxiv'}, 'type': 'function'}, {'id': 'call_iFVDUymIiytFagujB8Z7udRp', 'function': {'arguments': '{"query": "latest Tweet of author"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 178, 'total_tokens': 232, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': None, 'id': 'chatcmpl-BsVFDb7KNneL4XWYHk9hPOD1tkgMo', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--2db0619f-b567-4433-a582-70257b50abd6-0', tool_calls=[{'name': 'arxiv', 'args': {'que

#### 🏗️ Activity #2:

Please write out the steps the agent took to arrive at the correct answer.

### Answer
1. The agent receives the input and determines that it needs to use the Arxiv tool, adding a tool call request to the state.
2. The decision node detects the tool call request and routes the state to the action node to execute the tool.
3. The result from the tool call is appended to the state, and the agent is invoked again to decide the next step.
4. This process repeats two more times, as the agent sequentially calls additional tools (e.g., searching for each author's latest Tweet using Tavily), each time updating the state and re-evaluating.
5. After the third cycle, the agent no longer requests a tool call; the decision node recognizes this and routes the state to the END node, completing the process.

# 🤝 Breakout Room #2

## Part 1: LangSmith Evaluator

### Pre-processing for LangSmith

To do a little bit more preprocessing, let's wrap our LangGraph agent in a simple chain.

In [18]:
def convert_inputs(input_object):
  return {"messages" : [HumanMessage(content=input_object["question"])]}

def parse_output(input_state):
  return input_state["messages"][-1].content

agent_chain_with_formatting = convert_inputs | simple_agent_graph | parse_output

In [19]:
agent_chain_with_formatting.invoke({"question" : "What is RAG?"})

"RAG can refer to different concepts depending on the context. Could you please specify whether you're asking about RAG in the context of project management, machine learning, or another field?"

### Task 1: Creating An Evaluation Dataset

Just as we saw last week, we'll want to create a dataset to test our Agent's ability to answer questions.

In order to do this - we'll want to provide some questions and some answers. Let's look at how we can create such a dataset below.

```python
questions = [
    "What optimizer is used in QLoRA?",
    "What data type was created in the QLoRA paper?",
    "What is a Retrieval Augmented Generation system?",
    "Who authored the QLoRA paper?",
    "What is the most popular deep learning framework?",
    "What significant improvements does the LoRA system make?"
]

answers = [
    {"must_mention" : ["paged", "optimizer"]},
    {"must_mention" : ["NF4", "NormalFloat"]},
    {"must_mention" : ["ground", "context"]},
    {"must_mention" : ["Tim", "Dettmers"]},
    {"must_mention" : ["PyTorch", "TensorFlow"]},
    {"must_mention" : ["reduce", "parameters"]},
]
```

#### 🏗️ Activity #3:

Please create a dataset in the above format with at least 5 questions.

In [20]:
# Questions and required mentions for validating the retrieval system

questions = [
    "Who are the main authors of the QLoRA paper?",
    "What optimizer is introduced or used in QLoRA?",
    "Which data type was proposed in the QLoRA paper?",
    "What is Retrieval Augmented Generation (RAG) in the context of QLoRA?",
    "Which deep learning frameworks are most commonly referenced in the QLoRA paper?",
    "What are the key improvements QLoRA offers over previous methods?"
]

answers = [
    {"must_mention": ["Tim", "Dettmers"]},
    {"must_mention": ["paged", "optimizer"]},
    {"must_mention": ["NF4", "NormalFloat"]},
    {"must_mention": ["ground", "context"]},
    {"must_mention": ["PyTorch", "TensorFlow"]},
    {"must_mention": ["reduce", "parameters"]},
]

Now we can add our dataset to our LangSmith project using the following code which we saw last Thursday!

In [21]:
from langsmith import Client

client = Client()

dataset_name = f"Retrieval Augmented Generation - Evaluation Dataset - {uuid4().hex[0:8]}"

dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Questions about the QLoRA Paper to Evaluate RAG over the same paper."
)

client.create_examples(
    inputs=[{"question" : q} for q in questions],
    outputs=answers,
    dataset_id=dataset.id,
)

{'example_ids': ['62b72bfa-8355-49ff-84a8-ddc91ca3d139',
  '246a6b17-c232-44b2-97b3-7199b0bac4bf',
  '3dd539b8-74b8-424f-b79e-16d7f480ae73',
  '0c3818a1-505a-43df-9c1f-aa579c8d4754',
  '566fa25d-1a02-4fc6-8720-6c99c1671990',
  '625bf9bd-2ad4-48cc-89cc-269f57fe2e97'],
 'count': 6}

#### ❓ Question #3:

How are the correct answers associated with the questions?

> NOTE: Feel free to indicate if this is problematic or not

## Answer
The association between questions and answers relies on their positions within their respective arrays—each answer corresponds to the question at the same index. 
While this is simple, it introduces a hidden dependency: if the order of either array changes or if an item is added/removed from one but not the other, the mapping breaks and answers may no longer match the correct questions. 
This approach can make the code error-prone and harder to maintain, as mismatches may not be immediately obvious and could lead to subtle bugs.

### Task 2: Adding Evaluators

Now we can add a custom evaluator to see if our responses contain the expected information.

We'll be using a fairly naive exact-match process to determine if our response contains specific strings.

In [23]:
from langsmith.evaluation import EvaluationResult, run_evaluator

@run_evaluator
def must_mention(run, example) -> EvaluationResult:
    prediction = run.outputs.get("output") or ""
    required = example.outputs.get("must_mention") or []
    score = all(phrase in prediction for phrase in required)
    return EvaluationResult(key="must_mention", score=score)

#### ❓ Question #4:

What are some ways you could improve this metric as-is?

> NOTE: Alternatively you can suggest where gaps exist in this method.

### Answer
The current evaluator relies on exact string matching, which makes it quite rigid and potentially unreliable. It will fail to recognize correct answers if they use synonyms, different word forms (e.g., singular vs. plural), or paraphrased language, even if the meaning is preserved. This can lead to false negatives where a valid answer is marked as incorrect simply due to wording differences.

#### Potential improvements:
1. Incorporate semantic similarity: Use embeddings (e.g., via an LLM or sentence transformer) to compare the predicted output and the required phrases, allowing for more flexible matching based on meaning rather than exact wording.
2. Implement fuzzy matching: Use techniques like Levenshtein distance or token overlap to allow for minor variations, typos, or different word forms.
3. Expand the set of acceptable phrases: Maintain a list of synonyms or alternative phrasings for each required concept.
4. Use LLM-based evaluators: Leverage a language model to judge whether the required information is present, which can capture context and intent more robustly.
 
Overall, moving beyond strict string matching to more semantic or context-aware evaluation methods will make the metric more robust and reflective of true answer quality.


Task 3: Evaluating

All that is left to do is evaluate our agent's response!

In [24]:
experiment_results = client.evaluate(
    agent_chain_with_formatting,
    data=dataset_name,
    evaluators=[must_mention],
    experiment_prefix=f"Search Pipeline - Evaluation - {uuid4().hex[0:4]}",
    metadata={"version": "1.0.0"},
)

View the evaluation results for experiment: 'Search Pipeline - Evaluation - 9946-903006ac' at:
https://smith.langchain.com/o/bbbdc2e8-da22-470d-aa79-cd31d5ea3160/datasets/b6443cd6-3f21-4ef9-bc2a-81e17bf7acef/compare?selectedSessions=c0624185-6559-46df-8144-849d80cfe464




0it [00:00, ?it/s]

In [37]:
experiment_results

<ExperimentResults Search Pipeline - Evaluation - c647-b0a166cc>

## Part 2: LangGraph with Helpfulness:

### Task 3: Adding Helpfulness Check and "Loop" Limits

Now that we've done evaluation - let's see if we can add an extra step where we review the content we've generated to confirm if it fully answers the user's query!

We're going to make a few key adjustments to account for this:

1. We're going to add an artificial limit on how many "loops" the agent can go through - this will help us to avoid the potential situation where we never exit the loop.
2. We'll add to our existing conditional edge to obtain the behaviour we desire.

First, let's define our state again - we can check the length of the state object, so we don't need additional state for this.

In [29]:
class AgentState(TypedDict):
  messages: Annotated[list, add_messages]

Now we can set our graph up! This process will be almost entirely the same - with the inclusion of one additional node/conditional edge!

#### 🏗️ Activity #5:

Please write markdown for the following cells to explain what each is doing.

In this code cell, we are initializing a new LangGraph StateGraph called `graph_with_helpfulness_check` using the `AgentState` type. 
We then add two nodes to the graph: 
- The "agent" node, which will handle the main model's reasoning and response generation (using the `call_model` function).
- The "action" node, which will handle tool use or external actions (using the `tool_node` function).
This setup forms the foundation for a more advanced agent workflow, where we can later add additional logic—such as helpfulness checks or loop limits—by connecting these nodes with conditional edges.

In [31]:
graph_with_helpfulness_check = StateGraph(AgentState)

graph_with_helpfulness_check.add_node("agent", call_model)
graph_with_helpfulness_check.add_node("action", tool_node)

Then, we specify the entry point for our LangGraph by calling `set_entry_point("agent")` on the `graph_with_helpfulness_check` object. 
This means that whenever the graph is executed, it will begin processing at the "agent" node. 

Setting the entry point is essential because it tells the graph where to start the workflow, ensuring that the agent's reasoning and response generation logic is the first step in the process. 

In [32]:
graph_with_helpfulness_check.set_entry_point("agent")

This section of the code defines the `tool_call_or_helpful` function, which determines the next step in the agent's workflow based on the current state. The function first checks if the most recent message includes a tool call; if so, it directs the workflow to the "action" node for tool execution. If not, it evaluates whether the conversation has exceeded a certain length, in which case it ends the process. Otherwise, it uses a language model to assess if the agent's latest response is extremely helpful. This is done by prompting the model to return 'Y' for helpful or 'N' for unhelpful. If the response is helpful, the workflow ends; if not, the agent continues reasoning or taking further actions. This logic enables the agent to decide dynamically whether to act, continue, or finish based on the helpfulness of its responses and the conversation's context.



In [34]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

def tool_call_or_helpful(state):
  last_message = state["messages"][-1]

  if last_message.tool_calls:
    return "action"

  initial_query = state["messages"][0]
  final_response = state["messages"][-1]

  if len(state["messages"]) > 10:
    return "END"

  prompt_template = """\
  Given an initial query and a final response, determine if the final response is extremely helpful or not. Please indicate helpfulness with a 'Y' and unhelpfulness as an 'N'.

  Initial Query:
  {initial_query}

  Final Response:
  {final_response}"""

  helpfullness_prompt_template = PromptTemplate.from_template(prompt_template)

  helpfulness_check_model = ChatOpenAI(model="gpt-4.1-mini")

  helpfulness_chain = helpfullness_prompt_template | helpfulness_check_model | StrOutputParser()

  helpfulness_response = helpfulness_chain.invoke({"initial_query" : initial_query.content, "final_response" : final_response.content})

  if "Y" in helpfulness_response:
    return "end"
  else:
    return "continue"

#### 🏗️ Activity #6:

Please write what is happening in our `tool_call_or_helpful` function!

The `tool_call_or_helpful` function is a conditional edge function used in our LangGraph agent workflow to determine the next step based on the agent's most recent message and the overall conversation state. Here's what happens inside the function:

1. It checks if the last message from the agent contains any tool calls. If so, it returns "action", indicating the agent should execute a tool.
2. If not, it checks if the conversation has gone on for more than 10 messages. If so, it returns "END" to stop the loop and finish the conversation.
3. Otherwise, it uses an LLM (specifically, a GPT-4.1-mini model) to evaluate whether the agent's final response is "extremely helpful" with respect to the initial user query. It does this by constructing a prompt that includes both the initial query and the final response, and asks the LLM to answer with 'Y' (helpful) or 'N' (not helpful).
4. If the LLM responds with 'Y', the function returns "end" to finish the conversation. If not, it returns "continue", which causes the agent to keep working on the task.

In [31]:
graph_with_helpfulness_check.add_conditional_edges(
    "agent",
    tool_call_or_helpful,
    {
        "continue" : "agent",
        "action" : "action",
        "end" : END
    }
)

The line `graph_with_helpfulness_check.add_edge("action", "agent")` adds a direct edge in the LangGraph workflow from the "action" node back to the "agent" node. 
This means that after the agent executes a tool (the "action" step), the workflow will return to the agent to continue processing the conversation or task. This loop allows the agent to iteratively use tools and refine its responses until a helpful final answer is produced or the conversation ends.

In [32]:
graph_with_helpfulness_check.add_edge("action", "agent")

Finally, the whole graph is compile.

In [36]:
agent_with_helpfulness_check = graph_with_helpfulness_check.compile()

This code demonstrates how to run the compiled LangGraph agent (`agent_with_helpfulness_check`) asynchronously with a user input. 

- The `inputs` dictionary contains a list of messages, starting with a `HumanMessage` that asks three related questions about machine learning topics.
- The `async for` loop streams updates from the agent as it processes the input. The `astream` method yields intermediate results (or "chunks") as the agent moves through the graph, such as generating responses, calling tools, or evaluating helpfulness.
- For each update, the code prints out which node in the workflow produced the update and displays the current messages at that step.


In [37]:
inputs = {"messages" : [HumanMessage(content="Related to machine learning, what is LoRA? Also, who is Tim Dettmers? Also, what is Attention?")]}

async for chunk in agent_with_helpfulness_check.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        print(values["messages"])
        print("\n\n")

Receiving update from node: 'agent'
[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_CugaV1YNy6S4tL1WVO5jj5ZS', 'function': {'arguments': '{"query": "LoRA machine learning"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}, {'id': 'call_UsgTAAtMV6CNmq0gufzmrrHs', 'function': {'arguments': '{"query": "Tim Dettmers"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}, {'id': 'call_JephcJVrvOp2BvjxiahyLbum', 'function': {'arguments': '{"query": "Attention in machine learning"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 79, 'prompt_tokens': 177, 'total_tokens': 256, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': None, 'id': 'chatcmpl-BsV

### Task 4: LangGraph for the "Patterns" of GenAI

Let's ask our system about the 4 patterns of Generative AI:

1. Prompt Engineering
2. RAG
3. Fine-tuning
4. Agents

In [38]:
patterns = ["prompt engineering", "RAG", "fine-tuning", "LLM-based agents"]

In [39]:
for pattern in patterns:
  what_is_string = f"What is {pattern} and when did it break onto the scene??"
  inputs = {"messages" : [HumanMessage(content=what_is_string)]}
  messages = agent_with_helpfulness_check.invoke(inputs)
  print(messages["messages"][-1].content)
  print("\n\n")

Prompt engineering is the process of designing and refining input prompts to effectively communicate with AI language models, such as GPT, to elicit accurate, relevant, and useful responses. It involves crafting prompts that guide the model's output in a desired direction, often through techniques like specifying context, framing questions clearly, or using specific keywords.

Prompt engineering has gained prominence with the rise of large language models (LLMs) around 2020 and 2021, as these models became more capable and widely adopted. The practice became especially important as users and developers sought to maximize the utility of LLMs across various applications, from chatbots to content generation.

Would you like me to find more detailed historical information or recent developments related to prompt engineering?



RAG, which stands for Retrieval-Augmented Generation, is a technique in natural language processing that combines pre-trained language models with information retri